In [10]:
from tqdm import tqdm
import re
import os
import numpy as np
import argparse
import json
import random
import jieba
import hanlp
import time
import pandas as pd
#import torch
import openpyxl
#from torch import cosine_similarity

#from text2vec import SentenceModel, cos_sim, semantic_search
from datetime import datetime
#从自写工具包中导入：修饰词判定、修饰词查找
from utils import modifier_judge, modifier_search

#device = torch.device('cuda')
#use_gpu = torch.cuda.is_available()

def read_vectors(path, topn):  # read top n word vectors, i.e. top is 10000
    lines_num, dim = 0, 0
    vectors = {}
    iw = []
    wi = {}
    #num_file = sum([1 for i in open(path, encoding='utf-8', errors='ignore')])  
    #print(num_file)
    with open(path, encoding='utf-8', errors='ignore') as f:
        first_line = True
        for line in tqdm(f):
            if first_line:
                first_line = False
                dim = int(line.rstrip().split()[1])
                continue
            lines_num += 1
            tokens = line.rstrip().split(' ')
            vectors[tokens[0]] = np.expand_dims(np.asarray([float(x) for x in tokens[1:]]),axis=0)
            #vectors[tokens[0]] = torch.tensor([float(x) for x in tokens[1:]], device=device).unsqueeze(0)
            iw.append(tokens[0])
            if topn != 0 and lines_num >= topn:
                break
    for i, w in enumerate(iw):
        wi[w] = i
    return vectors, iw, wi, dim

#Use your pre-trained file and put it in data/pre_trained_vactors
vectors_path = "../data/pre-trained_vectors/sgns.renmin.bigram"
topn = 0
vectors, iw, wi, dim = read_vectors(vectors_path, topn) 


355990it [00:14, 25332.51it/s]


In [11]:
def cos_sim(vec1, vec2):
    if vec1.all() == 0 or vec2.all() == 0:
        return np.zeros([1])[0]
    else:
        return (vec1.dot(vec2[0]) / (np.linalg.norm(vec1[0]) * np.linalg.norm(vec2[0])))[0]



def word2vec(word1):
    if word1 in vectors:
        return vectors[word1]
    else:
        #print('not find')
        first_line = True
        for character in word1:
            if first_line:
                first_line = False
                if character in vectors:
                    vector = vectors[character]
                else: 
                    vector = np.zeros([1, 300])
                continue
            if character in vectors:
                vector = np.append(vector,vectors[character], axis = 0)
        return np.mean(vector, axis = 0).reshape(1,300)

def words2vec(word_list):
    first_line = True
    for word in word_list:
        if first_line:
            first_line = False
            vector = word2vec(word)
            continue
        vector =  np.append(vector,word2vec(word), axis = 0)
    return np.mean(vector, axis = 0).reshape(1,300)



1.0000000000000002
000
-0.5


In [12]:

def read_phrases(path):
    with open (path, 'r',  encoding='utf-8') as phrases_read:
        phrases = []
        first_line = True
        for line in phrases_read:
            if first_line:
                first_line = False
                line_number = line
                continue
            cut = line.replace('\n', '').split('。')
            phrase = cut[0].split(',')
            sentence = cut[1]+' '+cut[2]
            phrase = [tuple(x.split(';')) for x in phrase]
            phrase.append(sentence)
            phrases.append(tuple(phrase))
    return phrases, line_number


def simlirity_for_phase_mode1(phase_before, phase_now):
    #print(phase_before, phase_now)
    sim_v = cos_sim(word2vec(phase_before[0]),word2vec(phase_now[0]))
    sim_key = cos_sim(word2vec(phase_before[1]),word2vec(phase_now[1]))
    sim_modify = np.zeros([1])[0]
    if len(phase_now)>2:
        if len(phase_before)>2:
            for word_now in phase_now[2:]:
                sim_modify_tem = np.zeros([1])[0]
                for word_before in phase_before[2:]:
                    tem_sim = cos_sim(word2vec(word_before),word2vec(word_now))
                    if tem_sim > sim_modify_tem:
                        sim_modify_tem = tem_sim
                sim_modify = sim_modify + sim_modify_tem
            sim_modify = sim_modify/len(phase_now[2:])
            return 0.1*sim_v+0.5*sim_key+0.4*sim_modify
        else:
            return 0.1*sim_v+0.6*sim_key
    else:
        return 0.14*sim_v+0.86*sim_key
    
def simlirity_for_phase_mode2(phase_before, phase_now):
    sim_key = cos_sim(word2vec(phase_before[0]),word2vec(phase_now[0]))
    sim_modify = np.zeros([1])[0]
    if len(phase_now)>1:
        if len(phase_before)>1:
            sim_modify = cos_sim(words2vec(phase_before[1:]),words2vec(phase_now[1:]))
            return 0.4*sim_key+0.6*sim_modify
        else:
            return 0.4*sim_key
    else:
        return sim_key

def simlirity_for_phrases(phrases_before, phrases_now):
    sum1 = np.zeros([1])[0]
    sum2 = np.zeros([1])[0]
    count1 = len(phrases_now[0])
    count2 = len(phrases_now[1])
    another_mode = False
    if len(phrases_now[1])> len(phrases_before[1]):
        another_mode = True
    temp_phrases_now = [list(i_now) for i_now in phrases_now]
    temp_phrases_before = [list(i_before) for i_before in phrases_before]
    #print(phrases_before, phrases_now)
    #print(temp_phrases_before, temp_phrases_now)
    if phrases_now[0][0] and phrases_now[1][0]:
        the_type = True
    else:
        the_type = False
    #现在的无mode2，过去无mode2
    #print(phrases_before, phrases_now)
    #print(phrases_before[1][0], phrases_now[1])
    while temp_phrases_now[0][0] and temp_phrases_before[0][0]:
        sim1_2 = np.zeros([1])[0]
        for index1_1, phrase_now in enumerate(temp_phrases_now[0]):            
            for index1_2, phrase_before in enumerate(temp_phrases_before[0]):
                sim_temp1 = simlirity_for_phase_mode1(phrase_before.split(' '), phrase_now.split(' '))
                #print(sim_temp1)
                if sim_temp1 > sim1_2:
                    pos1_1 = phrase_now
                    pos1_2 = phrase_before
                    sim1_2 = sim_temp1
        sum1 = sum1 + sim1_2
        if len(temp_phrases_now[0]) == 1 or len(temp_phrases_before[0]) == 1:
            break
        temp_phrases_now[0].remove(pos1_1)
        temp_phrases_before[0].remove(pos1_2)
    
    while temp_phrases_now[1][0] and temp_phrases_before[1][0]:
        sim2_2 = np.zeros([1])[0]-0.5
        pos2_1 = ''
        pos2_2 = ''
        for index2_1, phrase_now2 in enumerate(temp_phrases_now[1]):
            for index2_2, phrase_before2 in enumerate(temp_phrases_before[1]):
                sim_temp2 = simlirity_for_phase_mode2(phrase_before2.split(' '), phrase_now2.split(' '))
                if sim_temp2 > sim2_2:
                    pos2_1 = phrase_now2
                    pos2_2 = phrase_before2
                    sim2_2 = sim_temp2
        sum2 = sum2 + sim2_2
        if len(temp_phrases_now[1]) == 1 or len(temp_phrases_before[1]) == 1:
            if another_mode and pos2_1 in temp_phrases_now[1]:
                temp_phrases_now[1].remove(pos2_1)
            break
        try:
            temp_phrases_now[1].remove(pos2_1)
            temp_phrases_before[1].remove(pos2_2)
        except:
            print(phrases_before, phrases_now)
            print(temp_phrases_before, temp_phrases_now, pos2_1, pos2_2)
            break
            
    if another_mode:
        while temp_phrases_now[1][0] and temp_phrases_before[0][0]:
            sim2_3 = np.zeros([1])[0]
            for index23_1, phrase_now23 in enumerate(temp_phrases_now[1]):
                for index23_2, phrase_before23 in enumerate(temp_phrases_before[0]):
                    sim_temp23 = simlirity_for_phase_mode2(phrase_before23.split(' ')[1:], phrase_now23.split(' '))
                    if sim_temp23 > sim2_3:
                        pos23_1 = phrase_now23
                        pos23_2 = phrase_before23
                        sim2_3 = sim_temp23
            sum2 = sum2 + sim2_3
            if len(temp_phrases_now[1]) == 1 or len(temp_phrases_before[0]) == 1:
                break
            try:
                temp_phrases_now[1].remove(pos23_1)
            except:
                print(phrases_before, phrases_now)
                print(temp_phrases_before, temp_phrases_now, pos23_1, pos23_2)
                break
    
    if count1 > 0 :
        sim1 = sum1/count1
    if count2 > 0 :
        sim2 = sum2/count2
    
    if the_type:
        return 0.77*sim1 + 0.23*sim2
    else:
        return sim1 + sim2
        
        
def similirity_for_corpus(corpus_before, corpus_now):
    inno_dict={ "α=0.88":[], "α=0.9":[]}
    sim_dict={ "α=0.88":[], "α=0.9":[]}
    defussion_dict = {"α=0.88":[], "α=0.9":[]}
    dire_dict = {"α=0.88":[], "α=0.9":[]}
    count_list = []

    for line_now in corpus_now:
     
        sim_dot88 = 0
        dot88 =True
        defu_dot88 = 'null'
        dir_dot88 = 'null'        
        sim_dot9 = 0
        dot9 =True
        defu_dot9 = 'null'
        dir_dot9 = 'null'        

        for line_before in corpus_before:
            common_elements = list(set(line_before[1][0].split(' ')).intersection(line_now[1][0].split(' ')))
            #print(line_now[-1]+' '+line_before[-1])
            if common_elements:
                temp_sim_dot = simlirity_for_phrases(line_before[2:-1], line_now[2:-1])
             
                if dot88:
                    if temp_sim_dot > sim_dot88:                            
                        sim_dot88 = temp_sim_dot
                        defu_dot88 = line_before[-1]+' '+line_now[-1]
                        dir_dot88 = line_before[0][0]+','+line_now[0][0]
                        if sim_dot88 >= 0.88:
                            dot88 = False
                if dot9:
                    if temp_sim_dot > sim_dot9:                            
                        sim_dot9 = temp_sim_dot
                        defu_dot9 = line_before[-1]+' '+line_now[-1]
                        dir_dot9 = line_before[0][0]+','+line_now[0][0]
                        if sim_dot9 >= 0.9:
                            dot9 = False                     
                            break
        
        if dot88 or dot9:
            for line_this in count_list:
                common_this_elements = list(set(line_this[1][0].split(' ')).intersection(line_now[1][0].split(' ')))
                if common_this_elements:
                    temp_this_sim_dot = simlirity_for_phrases(line_this[2:-1], line_now[2:-1])
                    if dot88:
                        if temp_this_sim_dot > sim_dot88:
                            sim_dot88 = temp_this_sim_dot
                            defu_dot88 = line_this[-1]+' '+line_now[-1]
                            dir_dot88 = line_this[0][0]+','+line_now[0][0]
                            if sim_dot88 >= 0.88:
                                dot88 = False 

                    if dot9:
                        if temp_this_sim_dot > sim_dot9:                            
                            sim_dot9 = temp_this_sim_dot
                            defu_dot9 = line_this[-1]+' '+line_now[-1]
                            dir_dot9 = line_this[0][0]+','+line_now[0][0]
                            if sim_dot9 >= 0.9:
                                dot9 = False                     
                                break
                                
        if sim_dot88 < 0.88:
            count_list.append(tuple(line_now))
            inno_dict["α=0.88"].append(line_now)
        
        sim_dict["α=0.88"].append(sim_dot88)
        defussion_dict["α=0.88"].append(defu_dot88)
        dire_dict["α=0.88"].append(dir_dot88)
        
        if sim_dot9 < 0.9:
            inno_dict["α=0.9"].append(line_now)
        sim_dict["α=0.9"].append(sim_dot9)
        defussion_dict["α=0.9"].append(defu_dot9)
        dire_dict["α=0.9"].append(dir_dot9)
        
        
        
    return sim_dict,defussion_dict, dire_dict, inno_dict, count_list
  

In [7]:
province_list = os.listdir('../data/pre_processed')
#province_list.remove('.DS_Store')
#file_name_list.remove('国务院')
print(province_list)



            
    

['安徽省', '山西省', '福建省', '上海市', '陕西省', '广西壮族自治区', '宁夏回族自治区', '重庆市', '浙江省', '贵州省', '湖北省', '湖南省', '青海省', '黑龙江省', '河北省', '河南省', '江西省', '天津市', '国务院', '辽宁省', '江苏省', '新疆维吾尔自治区', '山东省', '甘肃省', '四川省', '海南省', '内蒙古自治区', '北京市', '云南省', '西藏自治区', '吉林省', '广东省']


In [13]:
#1119，该这里了
for province in province_list[0:2]:
    first_time = True
    denominator_selfCorpus = []
    #for year in time_list[1:]:
    for time_year in range(1998,2025):  
        save_path = "../data/sig_reports/"+province+str(time_year)+"年政府工作报告.txt"
        if os.path.exists(save_path):
            selfCorpus, num =read_phrases(save_path)
            #if first_time:
             #   first_time = False
            #    denominator_selfCorpus.extend(selfCorpus)
             #   continue
            print(province, time_year)
            simi_list,def_list, dir_list, inno_list,count_list = similirity_for_corpus(denominator_selfCorpus, selfCorpus)
            denominator_selfCorpus.extend(count_list)
            print(len(denominator_selfCorpus))
            for alpha in ["α=0.88"]: 
                data_dict = {"sim": simi_list[alpha], "sentences": def_list[alpha], "from":dir_list[alpha]}
                df = pd.DataFrame(data_dict)
                excel_path = "../data/adoption_judge/"+province+str(time_year)+"年政府工作报告_self_similarity.xlsx"
                df.to_excel(excel_path,index=False)

                with open(file ="../data/adoptions/"+province+str(time_year)+"年政府工作报告_self_similarity.txt", mode='w',encoding='utf - 8') as f_forward:
                    f_forward.write('innovation_num:  '+str(len(inno_list[alpha]))+num+'\n')
                    for every_line in inno_list[alpha]:
                        f_forward.write(every_line[0][0]+',')
                        f_forward.write(every_line[1][0]+',')
                        for index_mode1_phrase, mode1_phrase in enumerate(every_line[2]):
                            for index_mode1_word, mode1_word in enumerate(mode1_phrase):
                                f_forward.write(mode1_word)
                            if index_mode1_phrase< len(every_line[2])-1:
                                f_forward.write(';')
                        f_forward.write(',')
                        for index_mode2_phrase, mode2_phrase in enumerate(every_line[3]):
                            for index_mode2_word, mode2_word in enumerate(mode2_phrase):
                                f_forward.write(mode2_word)
                            if index_mode2_phrase< len(every_line[3])-1:
                                f_forward.write(';') 
                        f_forward.write('。')
                        f_forward.write(every_line[4])
                        f_forward.write('\n')

安徽省 1998
709
安徽省 1999
1301
563
山西省 1999
1092
山西省 2000
1525
山西省 2001
2012
山西省 2002
2476
山西省 2003
2898
山西省 2004
3445
山西省 2005
3839
山西省 2006
4317
山西省 2007
4754
山西省 2008
5216
山西省 2009
5703
山西省 2010
6034
山西省 2011
6372
山西省 2012
6663
山西省 2013
6979
山西省 2014
7348
山西省 2015
7713
山西省 2016
8135
山西省 2017
8858
山西省 2018
9506
山西省 2019
10070
山西省 2020
10629
山西省 2021
11138
山西省 2022
11647
山西省 2023
12319
山西省 2024
12819


In [17]:
import pandas as pd
df_Annotation_auto = pd.read_excel('../中国地方政府工作报告数据库/地级市政府工作报告/preprocessed_data/df_Annotation.xlsx', index_col=0)
for index, row in df_Annotation_auto.iterrows():
    s1 = row['sentence1']
    s2 = row['sentence2']
    label = row['label']
    break
print(s1, s2, label)

强高技能人才队伍建设 努力培养和壮大党政人才队伍 1


In [18]:
import os
from openai import OpenAI
from pathlib import Path

#list1 = []
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-244031ceb83a4722a1abe204942fb30b", # 如何获取API Key：https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

import pandas as pd
df_Annotation_auto = pd.read_excel('../中国地方政府工作报告数据库/地级市政府工作报告/preprocessed_data/df_Annotation.xlsx', index_col=0)
for index, row in df_Annotation_auto[:10].iterrows():
    s1 = row['sentence1']
    s2 = row['sentence2']
    label = row['label']
    completion = client.chat.completions.create(
        model="qwen-plus-latest", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=[
            {"role": "system", "name": "annotator", 
             "content": '你是一个 <annotator>, 也是一位在中国政府政策研究方面的专家。 您需要基于 <user>提供的来自中国政府工作报告中的两个代表政策举措的句子和对应的判断，基于五项原则进行校对。判断的label是0代表sentence2相对于sentence1是非创新，1则是创新。原则1，这些句子往往没有主语，以由动词、修饰词和宾语组成；在创新性比较中宾语最为重要，对政策举措的含义有决定性影响，如果两个句子的宾语含义差异较大，或者后一句子的并列宾语明显多于前者且含义超出前一句子的宾语，那么后一句子肯定相对于前者是创新的；其次重要的是直接修饰宾语的修饰词，再次是修饰词的修饰词，依次递减。原则2，句子中的动词是无关的，其含义不影响创新性判断结果，例如sentence1:加强城市化建设; sentence2:重视城市化建设,则sentence2相对于sentence1不是创新，因为"重视"和"加强"对结果没有影响。原则3，如果前一个政策举措的含义只是后一个政策举措的子集，例如，sentence1:做好民生工作; sentence2:做好民生、防灾工作，这种情况下后一个句子（即sentence2）相对于前一个句子（即sentence1）是创新的。原则4，如果两个句子的内容和结构完全不同，没有直接的可比性，则默认后一句子相对于前一句子是创新的。原则5，如果后一个句子的字数明显少于前一个句子，且核心单词都被前一个句子包含，那么后一个句子相对前一个句子不是创新，例如entence1:加强农村实用人才队伍建设; sentence2:加强人才队伍建设,则sentence2相对于sentence1不是创新。'},
            {'role': 'user', 'content': '请指出对sentence2相对于sentence1的错误判断。请注意，label为1则判断结果为创新，label为2则判断结果为非创新'},
            {'role': 'user', 'content': '如果某条记录的`label`标注为1但实际上应该是0，或者`label`标注为0但实际上应该是1，这些都属于错误判断。'},
            {"role": "user", "content": "判断这一标注是否正确，并以最终结论输出“这一标注正确”或“这一标注错误”：sentence1:"+s1+"; sentence2:"+s2+"; lanel:"+str(label)+"。"}
            ]
    )


    print(completion.choices[0].message.content)
    #if "这一标注正确" in completion.choices[0].message.content:
    #    print('yes')
    #else:
    #    print('no')
    #    print()
#print(list1)

**这一标注正确。**

### 分析如下：

- **sentence1**: “强高技能人才队伍建设”  
  → 宾语是“高技能人才队伍”，动词是“强”（即“加强”），修饰词为“高技能”。

- **sentence2**: “努力培养和壮大党政人才队伍”  
  → 宾语是“党政人才队伍”，动词为“培养和壮大”，修饰词为“党政”。

### 原则分析：

- **原则1（宾语为核心）：**
  - sentence1 的宾语是“高技能人才队伍”，强调的是技术性强、操作层面的人才；
  - sentence2 的宾语是“党政人才队伍”，强调的是政府机关、党务管理方面的人才。
  - 两者在宾语的含义上差异明显，指向完全不同类别的“人才队伍”。

- **原则3（是否为子集关系）：**
  - sentence2 并不是 sentence1 的扩展或包含，二者内容不重叠，不存在子集关系。

- **原则4（不可比时默认为创新）：**
  - 两句话内容和对象有本质区别，不具备直接延续性，属于不同政策方向。

- **原则5（字数与核心词问题）：**
  - 不适用，因为sentence2并没有在表达上缩减，且其核心概念“党政人才队伍”与sentence1的“高技能人才队伍”不属于同一范畴。

### 结论：

由于两个句子所指的政策对象完全不同（一个是“高技能人才”，一个是“党政人才”），因此 sentence2 相对于 sentence1 是具有创新性的政策举措。

✅ **这一标注正确（label=1）**
这一标注错误。

**分析过程：**

根据原则1，政策举措的**宾语**是判断创新性的最关键因素。  
- sentence1的宾语是“党政人才队伍”，强调的是**党政领域的人才培养和壮大**；
- sentence2的宾语是“高技能人才队伍”，强调的是**具有高技能的专业人才队伍建设**。

两者宾语在内容和指向上有明显差异，分别聚焦于不同的群体：“党政人才” vs “高技能人才”。因此，sentence2在宾语上与sentence1有实质性不同，不是其子集，也不是其延续或扩展。

再依据原则3，只有当前一句子的含义是后一句子的一部分时，才可以判断为创新；而这里两句之间没有包含关系。

因此，sentence2相对于sentence1并不是创新，而是

In [3]:
import os
from pathlib import Path
from openai import OpenAI

client = OpenAI(
    api_key="sk-244031ceb83a4722a1abe204942fb30b", # 如何获取API Key：https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope服务base_url
)

file_object = client.files.create(file=Path("../中国地方政府工作报告数据库/地级市政府工作报告/preprocessed_data/df_Annotation_auto.xlsx"), purpose="file-extract")
print(file_object.model_dump_json())


{"id":"file-fe-a6725d7996d3485ca10c1e17","bytes":2490580,"created_at":1748009193,"filename":"df_Annotation_auto.xlsx","object":"file","purpose":"file-extract","status":"processed","expires_at":null,"status_details":null}


In [9]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="sk-244031ceb83a4722a1abe204942fb30b",  # 如果您没有配置环境变量，请在此处替换您的API-KEY
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 填写DashScope服务base_url
)
# 初始化messages列表
completion = client.chat.completions.create(
    model="qwen-long",
    messages=[
        {'role': 'system', "content": '你是一个 <judger>, 也是一位在中国政策研究方面的专家。你需要基于 <user>提供的excel文件列出其中的错误判断。文件中是标注过的正确分类，其中label是0代表sentence2相对于sentence1是非创新，1则是创新。原则1，这些句子往往没有主语，以由动词、修饰词和宾语组成；在创新性比较中宾语最为重要，对政策举措的含义有决定性影响，如果两个句子的宾语含义差异较大，或者后一句子的并列宾语明显多于前者且含义超出前一句子的宾语，那么后一句子肯定相对于前者是创新的；其次重要的是直接修饰宾语的修饰词，再次是修饰词的修饰词，依次递减。原则2，句子中的动词是无关的，其含义不影响创新性判断结果，例如sentence1:加强城市化建设; sentence2:重视城市化建设,则sentence2相对于sentence1不是创新，因为"重视"和"加强"对结果没有影响。原则3，如果前一个政策举措的含义只是后一个政策举措的子集，例如，sentence1:做好民生工作; sentence2:做好民生、防灾工作，这种情况下后一个句子（即sentence2）相对于前一个句子（即sentence1）是创新的。原则4，如果两个句子的内容和结构完全不同，没有直接的可比性，则默认后一句子相对于前一句子是创新的。如果后一个句子的字数明显少于前一个句子，且核心单词都被前一个句子包含，那么后一个句子相对前一个句子不是创新，例如entence1:加强农村实用人才队伍建设; sentence2:加强人才队伍建设,则sentence2相对于sentence1不是创新。'},
        # 请将 'file-fe-xxx'替换为您实际对话场景所使用的 file-id。
        {'role': 'system', 'content': 'fileid://file-fe-a6725d7996d3485ca10c1e17'},
        {'role': 'user', 'content': '请指出对sentence2相对于sentence1的错误判断。请注意，label为1则判断结果为创新，label为2则判断结果为非创新'},
        {'role': 'user', 'content': '如果某条记录的`label`标注为1但实际上应该是0，或者`label`标注为0但实际上应该是1，这些都属于错误判断。'},
        {'role': 'user', 'content': '例如，Sentence1: 落实职务待遇与业务支出管理规定\n  - Sentence2: 出台省政府立法工作规定 - label: 1, 这个就是正确标注，因为两个句子的主题完全不同，Sentence2涉及的是立法工作，而Sentence1涉及的是职务待遇和业务支出管理'}
    ],
    stream=True,
    stream_options={"include_usage": True}
)

full_content = ""
for chunk in completion:
    if chunk.choices and chunk.choices[0].delta.content:
        # 拼接输出内容
        full_content += chunk.choices[0].delta.content
        #print(chunk.model_dump())

print({full_content})

{'根据您提供的原则和示例，我将检查表格中的每一项，以确定是否存在错误的创新性判断。以下是详细的分析和错误判断的列表：\n\n### 错误判断示例：\n\n#### 1. **Sheet1**\n| Index | Sentence1 | Sentence2 | Label | 评注 |\n|-------|------------|------------|-------|------|\n| 0     | 落实职务待遇与业务支出管理规定 | 出台省政府立法工作规定 | 1 | 正确标注。两个句子的主题完全不同，Sentence2涉及的是立法工作，而Sentence1涉及的是职务待遇和业务支出管理。 |\n| 1     | 推进创新型广东建设大力推进自主创新 | 深化法治建设四级同创 | 1 | 错误标注。两个句子的主题不同，Sentence1强调的是创新和自主创新，而Sentence2强调的是法治建设。应标注为0。 |\n| 2     | 支持商品房市场更好满足购房者的合理住房需求 | 视同仁满足不同所有制房地产企业的合理融资需求 | 1 | 错误标注。虽然都提到“满足合理需求”，但Sentence1侧重于购房者的需求，而Sentence2侧重于房地产企业的融资需求，属于不同层面的需求。应标注为0。 |\n| 3     | 企一策推动省属国企战略性重组 | 链一策拓长补短 | 1 | 错误标注。两个句子的主题不同，Sentence1是关于国企重组，而Sentence2是关于供应链的策略。应标注为0。 |\n| 4     | 高质量谋划实施第二批授权事项清单 | 梳理细化超500项高频民生诉求事项职责清单 | 1 | 正确标注。Sentence2在Sentence1的基础上增加了具体细节，是创新。 |\n| 5     | 集中供养孤儿基本生活最低养育标准从每人每月1883元提高到1949元 | 集中供养孤儿基本生活保障省定最低标准从每人每月2017元提高到2295元 | 1 | 正确标注。Sentence2在Sentence1的基础上增加了具体细节，是创新。 |\n| 6     | 其中食用农产品批发市场实现全覆盖快检 | 食用农产品快检超800万批次 | 1 | 正确标注。Sentence2在Sentence1的基础上增加了具体细节，是创新。 |\